In [ ]:
# main.py or a cell in main.ipynb

from vrptw_parser import parse_homberger_1999_vrptw_file
from constructors import clarke_wright_vrptw
from utils import build_distance_matrix, solution_cost, is_feasible
from local_search import local_search_vrptw
from tabu_search import tabu_search_vrptw
from plot_utils import save_solution, plot_solution_from_file

# 1) Load instance
inst = parse_homberger_1999_vrptw_file("Vrp-Set-HG/C1_2_2.txt")

# 2) Build distance matrix
D = build_distance_matrix(inst)

# 3) Construct initial solution (Clarke–Wright)
routes_cw = clarke_wright_vrptw(inst, D=D)
print(f"CW cost: {solution_cost(routes_cw, D):.4f}")

# (Optional) polish with local search before Tabu
routes_ls = local_search_vrptw(
    inst,
    routes_cw,
    D=D,
    max_iterations=5_000,
    use_inter_route=True,
    time_limit_sec=10.0,
    verbose=True,
)
print(f"LS cost: {solution_cost(routes_ls, D):.4f}")

# # 4) Run Tabu Search starting from LS (or directly from CW)
# routes_ts = tabu_search_vrptw(
#     inst,
#     routes_ls,
#     D=D,
#     max_iterations=2_000,
#     tenure=15,                 # tune: 5–30 is a reasonable range
#     use_intra_relocate=True,
#     use_intra_swap=True,
#     use_2opt=True,
#     use_inter_relocate=True,
#     use_inter_swap=True,
#     time_limit_sec=230.0,       # safety cap
#     verbose=True,
# )

# # 5) Check & report
# final_cost = solution_cost(routes_ts, D)
# print(f"Tabu cost: {final_cost:.4f}")
# print("Feasible:", is_feasible(routes_ts, inst, D, require_all_customers=True))

# # 6) Save & plot
# sol_path = save_solution(inst, routes_ts, D)
# png_path = plot_solution_from_file(inst)

# print("Solution written to:", sol_path)
# print("Plot written to:", png_path)


In [ ]:
# # 7) (Optional) Run Iterated Local Search starting from the LS solution
# from ils import iterated_local_search_vrptw

# routes_ils = iterated_local_search_vrptw(
#     inst,
#     initial_routes=routes_ls,
#     D=D,
#     max_iterations=60,
#     perturb_strength=3,
#     max_perturb_strength=12,
#     adapt_perturb_every=5,
#     exploratory_accept_prob=0.15,
#     accept_equal_cost=True,
#     time_limit_sec=150.0,
#     seed=42,
#     ls_max_iterations=5000,
#     ls_use_inter_route=True,
#     verbose=True,
# )

# print("ILS cost:", solution_cost(routes_ils, D))
# print("Feasible:", is_feasible(routes_ils, inst, D, require_all_customers=True))
# print("Time to best (s):", getattr(iterated_local_search_vrptw, "time_to_best", None))

# # (Optional) save/plot ILS solution
# # sol_path_ils = save_solution(inst, routes_ils, D)
# # print("ILS solution written to:", sol_path_ils)


In [ ]:
# # 8) Run ALNS starting from the LS solution
# from alns import alns_vrptw

# routes_alns = alns_vrptw(
#     inst,
#     initial_routes=routes_ls,
#     D=D,
#     max_iterations=500,
#     remove_fraction_min=0.05,
#     remove_fraction_max=0.20,
#     ls_max_iterations=3000,
#     ls_use_inter_route=True,
#     apply_ls_every=1,
#     time_limit_sec=150.0,
#     seed=42,
#     verbose=True,
# )

# print("ALNS cost:", solution_cost(routes_alns, D))
# print("Feasible:", is_feasible(routes_alns, inst, D, require_all_customers=True))
# print("Time to best (s):", getattr(alns_vrptw, "time_to_best", None))

# # (Optional) save/plot ALNS solution
# # sol_path_alns = save_solution(inst, routes_alns, D)
# # print("ALNS solution written to:", sol_path_alns)


In [ ]:
# lns_vrptw(
#     inst: VRPTWInstance,
#     initial_routes: Optional[Sequence[Sequence[int]]] = None,
#     D: Optional[np.ndarray] = None,
#     max_iterations: int = 500,
#     remove_fraction_min: float = 0.05,
#     remove_fraction_max: float = 0.20,
#     use_shaw: bool = True,
#     use_worst: bool = True,
#     apply_ls_every: int = 1,
#     ls_max_iterations: int = 3000,
#     ls_use_inter_route: bool = True,
#     exploratory_accept_prob: float = 0.1,
#     accept_equal_cost: bool = True,
#     time_limit_sec: Optional[float] = None,
#     seed: Optional[int] = None,
#     verbose: bool = False,
# )

In [6]:
from lns import lns_vrptw

routes_lns = lns_vrptw(
    inst,
    initial_routes=routes_ls,
    D=D,
    max_iterations=500,
    remove_fraction_min=0.05,
    remove_fraction_max=0.20,
    use_shaw=True,
    use_worst=True,
    apply_ls_every=1,
    ls_max_iterations=3000,
    ls_use_inter_route=True,
    exploratory_accept_prob=0.1,
    accept_equal_cost=True,
    time_limit_sec=150.0,
    seed=42,
    verbose=True,
)
print("LNS cost:", solution_cost(routes_lns, D))
print("Feasible:", is_feasible(routes_lns, inst, D, require_all_customers=True))
print("Time to best (s):", getattr(lns_vrptw, "time_to_best", None))

[LNS] Start: cost=3521.9648, routes=27
[LNS] Iter 1: new best 3519.2731, routes=27, ruin=random, repair=greedy
[LNS] Iter 2: new best 3502.5634, routes=26, ruin=shaw, repair=greedy
[LNS] Iter 3: new best 3485.0500, routes=26, ruin=worst, repair=regret2
[LNS] Iter 10: new best 3306.8259, routes=26, ruin=worst, repair=greedy
[LNS] Iter 11: new best 3258.3617, routes=26, ruin=worst, repair=regret2
[LNS] Iter 12: new best 3250.2204, routes=26, ruin=worst, repair=greedy
[LNS] Iter 13: new best 3248.0180, routes=26, ruin=shaw, repair=regret2
[LNS] Iter 17: new best 3245.6770, routes=26, ruin=shaw, repair=greedy
[LNS] Iter 18: new best 3209.7047, routes=25, ruin=random, repair=greedy
[LNS] Iter 19: new best 3205.7795, routes=25, ruin=random, repair=regret2
[LNS] Iter 21: new best 3136.0836, routes=24, ruin=worst, repair=regret2
[LNS] Iter 22: new best 3129.8557, routes=24, ruin=worst, repair=greedy
[LNS] Iter 31: new best 3126.4409, routes=24, ruin=random, repair=greedy
[LNS] Iter 34: new bes